# Projet phylogénétique
Marine Djaffardjy (marine.djaffardjy@u-psud.fr)- Théophile Sanchez - Sarah Cohen Boulakia

------

## Introduction

Au cours de ce projet, vous étudierez trois espèces disparues de félins qui vivaient autrefois sur le continent Américain. Ces trois espèces, le _smilodon_ (tigre à dents de sabre), l'_homotherium_ (_scimitar toothed tigers_) et _M. trumani_ (guépard américain) se sont éteintes il y a environ 13 000 ans, à la fin de la dernière période glaciaire. Des séquences ADN partielles de la protéine cytochrome b de ces espèces ont pu être séquencées et vont vous permettre de retrouver les liens de parentés entre ces espèces et des espèces de félins contemporaines : le chat domestique, le lion, le léopard, le tigre, le puma, le guépard et les chats sauvages africains, chinois et européens. Sont aussi présentes dans le jeu de donnée des séquences issues d'espèces extérieures aux félins.

Afin de reconstruire l'arbre phylogénétique de ces espèces, vous utiliserez une méthode basée sur le calcul des distances évolutives entre les séquences ADN des protéines. Sachez qu'une démarche similaire peut-être appliquée aux séquences d'acides aminés.

Les différentes étapes qui vous permetterons de construire l'arbre sont détaillées dans ce notebook. Vous devrez implémenter les algorithmes en Python et répondre aux questions dans les cellules prévues.

Quelques conseils :
- Utiliser au maximum les fonctions présentes dans les packages de python (sauf si il vous est explicitement demandé de les réimplémenter). Si un problème vous paraît courant, il existe surement déjà une fonction pour le résoudre. Pour ce projet vous serez limité aux packages de base, à Numpy et ETE (seulement pour l'affichage des arbres).
- Si une partie de votre code ne vous semble pas très explicite, ajoutez des commentaires pour l'expliquer. Une personne qui lit votre code doit pouvoir comprendre son fonctionnement facilement.
- N'hésitez pas à chercher dans la documentation et sur internet. Cependant, faites attention au plagiat !

Le projet est à rendre **en binôme** par mail. Vous regrouperez votre notebook et les fichiers nécessaires à son fonctionnement dans une archive portant vos noms et prénoms.

------
## Importation des séquences

Le format FASTA permet de stocker plusieurs séquences (ADN, ARN ou peptidiques) dans un fichier. Les séquences que vous allez étudier ont été regroupées dans le fichier `cat_dna.fasta`.

**Exercice 1 :** Écriver une fonction permettant d'importer un fichier au format fasta et de le stocker dans un dictionnaire. Les clés seront les noms des séquences et les valeurs du dictionnaire seront les séquences d'adn.

In [1]:
def importe_fasta(nom):
    dicti={}
    o = open(nom,'r')
    r = o.read()
    groupes = r.split("\n\n")
    for groupe in groupes:
        groupe_separe =groupe.split("\n",1)
        cle= groupe_separe[0]
        donnee = groupe_separe[1].strip()
        dicti[cle] = donnee
    return dicti

monDi = importe_fasta('fasta/cat_dna.fasta')

for valeur in monDi:
    print(valeur)
    print(monDi[valeur])

>Sabertooth DNA (Smilodon)
CTAATTAAAATTATCAACCACTCATTCATTGATTTACCCACCCCATCCAACATTTCAGCA
TGATGAAACTTCGGCTCCTTATTAGGAGTGTGCTTAATCTTACAAATCCTCACTGGCTTA
TTTCTAGCCATACATTATACACCAGATACAACAACCGCCTTCTCATCAGTTACCCACATT
TGCCGTGATGTTAATTACGGCTGAATTATCCGATATATACACGCCAATGGAGCTTCCATA
TTCTTCATCTGCCTATATATACATGTAGGTCGAGCATATACTAC
>Homotherium DNA
CTAATTAAAATCATCAACCAATCATTCATTGACTTACCTACCCCCTCCAACATCTCAGCA
TGATGAAACTTCGGATCCCTACTAGGCATTTGCCTAATTCTTCAAATCCTCACAGGCTTA
TTCCTAGCCATACACTACACATCAGACACAACAACTGCTTTCTCATCAATCGCCCATATT
TGCCGTGACGTAAATTATGGTTGAATTATCCGATATATACACGCCAATGGAGCCTCTATA
TTCTTCATCTGTCTATACCTACATGTAGCTCGAGAATTTATTAC
>American Cat DNA (Miracinonyx)
CTTATTAAAATCATTAATCACTCATTCATTGATCTACCCACCCCATCCAACATTTCAGCA
TGATGAAACTTCGGTTCCCTACTAGGGGTCTGCCTAATCCTACAAATCCTAACCGGCCTC
TTCCTGGCTATACACTACACATCAGACACAATAACCGCCTTTTCATCAGTTACTCACATC
TGTCGTGACGTCAATTACGGCTGAATTATTCGGTATATACACGCCAACGGAGCCTCCATA
TTCTTTATCTGCCTATACATGCACGTAGGGCGAGAATATATTAC
>Spotted Hyena DNA
CTCATTAAAATTATCAACAAATCATTCATTGACCTCCC

------
## Alignement des séquences

La méthode que vous utiliserez pour calculer l'arbre phylogénétique nécessite de calculer la distance évolutive entre les séquences. Avant de pouvoir les calculer, il faut d'abord aligner les séquences en considérant trois types de mutations :
- les substitutions (un nucléotide est remplacé par un autre)
- les insertions
- les délétions
Par exemple, les séquences "ACTCCTGA" et "ATCTCGTGA" ont plusieurs alignements possibles : 

$A_1$ :
```
-ACTCCTGA
ATCTCGTGA
```

$A_2$ :
```
A-CTCCTGA
ATCTCGTGA
```

$A_3$ :
```
AC-TCCTGA
ATCTCGTGA
```
.

.

.

Le "-" désigne un *gap*, c'est à dire un "trou" dans l'alignement qui a été causé par une insertion ou une déletion. On regroupe ces deux types de mutations sous le terme indel.

Ces alignements correspondent à une multitude d'histoires phylogénétiques différentes. Pour sélectionner le meilleur alignement il faut donc introduire l'hypothèse du maximum de parcimonie qui privilégie l'histoire phylogénétique qui implique le moins d'hypothèses et donc, le moins de changements évolutifs. Par exemple, parmis les trois alignements ci-dessus on preferera l'alignement 2 car il correspond au scénario avec le moins de mutations:
- l'alignement 1 implique au minimum 1 indel et 2 substitutions
- l'alignement 2 implique au minimum 1 indel et 1 substitutions
- l'alignement 3 implique au minimum 1 indel et 2 substitutions

On peut maintenant définir un score d'identité que l'on va augmenter de 1 lorsque qu'il n'y pas eu de mutation et ainsi obtenir la matrice suivante :

|   &nbsp;   | A | C | G | T | - |
|   -   | - | - | - | - | - |
| **A** | 1 | 0 | 0 | 0 | 0 |
| **C** | 0 | 1 | 0 | 0 | 0 |
| **G** | 0 | 0 | 1 | 0 | 0 |
| **T** | 0 | 0 | 0 | 1 | 0 |
| **-** | 0 | 0 | 0 | 0 | 0 |

Cette matrice correspond au modèle d'évolution de l'ADN défini par Jukes et Cantor qui fait l'hypothèse d'un taux de mutation équivalent pour chacun des nucléotides. Cependant, en réalité ces taux ne sont pas les mêmes partout, on sait par exemple que le taux de transition (substitution A$\leftrightarrow$G ou C$\leftrightarrow$T) est différent du taux de transversions (substitution A$\leftrightarrow$T, C$\leftrightarrow$G, C$\leftrightarrow$A ou G$\leftrightarrow$T) et que d'autres facteurs devrait être pris en compte comme la fréquence du nucléotide dans l'ADN. [C'est pour cette raison qu'il existe beaucoup de modèles différents d'écrivant l'évolution de l'ADN.](https://en.wikipedia.org/wiki/Models_of_DNA_evolution) Dans la suite de ce projet nous utiliserons la matrice de similarité $S$ suivante : 

|   &nbsp;   | A  | C  | G  | T  | -  |
|   -   | -  | -  | -  | -  | -  |
| **A** | 10 | -1 | -3 | -4 | -5 |
| **C** | -1 | 7  | -5 | -3 | -5 |
| **G** | -3 | -5 | 9  | 0  | -5 |
| **T** | -4 | -3 | 0  | 8  | -5 |
| **-** | -5 | -5 | -5 | -5 | 0  |

**Exercice 2 :** Écriver la fonction permettant de calculer le score entre deux alignements avec la matrice de similarité précédente puis afficher le score des trois alignements $A_1$, $A_2$ et $A_3$. La classe permettant d'importer une matrice et de calculer le score entre deux lettres vous est déjà fournie, la matrice de similarité est stockée dans le fichier `dna_matrix` :


In [2]:
import numpy as np


class SimilarityMatrix:
    def __init__(self, filename):
        with open(filename) as f:
            self.letters = f.readline().split()
            self.values = np.loadtxt(filename, skiprows=1, usecols=range(1, len(self.letters) + 1))
        
    def score(self, letter1, letter2): # return the similarity score between letter1 and letter2
        return self.values[self.letters.index(letter1)][self.letters.index(letter2)]
    
# Example
similarity_matrix = SimilarityMatrix('dna_matrix')
print('Score between G and C:', similarity_matrix.score('G', 'C'))
print('Score between A and a gap:', similarity_matrix.score('-', 'A'))

Score between G and C: -5.0
Score between A and a gap: -5.0


In [3]:
def score_aligns(self,a1,a2):
    score = 0
    #for letter1,i in enumerate(list(a1)): # soit ça soit la méthode de i plus bas
    i=-1
    for letter in list(a1):
        i+=1
        score += self.values[self.letters.index(letter)][self.letters.index(list(a2)[i])]
    print(score)
    
score_aligns(similarity_matrix,'ATTC','ATAC')

21.0


------
### Algorithme de Needleman-Wunsch

Maintenant que vous avez vu ce qu'est une matrice de similarité et comment calculer le score de similarité d'un alignement, vous allez devoir implémenter un algorithme permettant de trouver le meilleur alignement global entre deux séquences. Avec deux séquences à aligner de taille $n$ et $m$, la première étape consiste à initialiser deux matrices de taille $(n+1) \times (m+1)$. La première est la matrice de score $M$ et la seconde sera la matrice de *traceback* $T$. 

Par exemple, avec la matrice $S$ et les séquences $A =$ "ACTCCTGA" et $B =$ "ATCTCGTGA", on initialise $M$ comme si l'on ajoutait des *gaps* partout :

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** | 0 |-5 |-10|-15|-20|-25|-30|-35|-40|-45|
| **A** |-5 | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** |-10| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** |-15| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** |-20| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** |-25| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** |-30| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **G** |-45| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **A** |-40| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; ||

Puis on initialise $T$ :

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** | o | l | l | l | l | l | l | l | l | l |
| **A** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **G** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **A** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; ||


Il faut ensuite remplir la matrice $M$ en suivant la formule $M_{ij} = \max(M_{i-1j-1} + s(A_i, B_j), M_{ij-1} + s(A_i, gap), M_{i-1j} + s(B_j,gap) )$ avec $i \in {2, \dots, n+1}$, $j \in {2, \dots, m+1}$ et la fonction $s$ qui calcule le score de similarité entre deux nucléotides. Pour chaque case de $T$ on remplie par :
- 'd' (*diagonal*) si $M_{ij}$ a été calculé en utilisant la diagonale $M_{i-1j-1}$,
- 'l' (*left*) si $M_{ij}$ a été calculé en utilisant la case de gauche $M_{ij-1}$,
- 'u' (*up*) si $M_{ij}$ a été calculé en utilisant la case du haut $M_{i-1j}$.

On obtient alors les matrices suivantes $M$ et $T$ : 

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** |  0| -5|-10|-15|-20|-25|-30|-35|-40|-45|
| **A** | -5| 10|  5|  0| -5|-10|-15|-20|-25|-30|
| **C** |-10|  5|  7| 12|  7|  2| -3| -8|-13|-18|
| **T** |-15|  0| 13|  8| 20| 15| 10|  5|  0| -5|
| **C** |-20| -5|  8| 20| 15| 27| 22| 17| 12|  7|
| **C** |-25|-10|  3| 15| 17| 22| 22| 19| 14| 11|
| **T** |-30|-15| -2| 10| 23| 18| 22| 30| 25| 20|
| **G** |-35|-20| -7|  5| 18| 18| 27| 25| 39| 34|
| **A** |-40|-25|-12|  0| 13| 17| 22| 23| 34| 49|

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** | o | l | l | l | l | l | l | l | l | l |
| **A** | u | d | l | l | l | l | l | l | l | d |
| **C** | u | u | d | d | l | d | l | l | l | l |
| **T** | u | u | d | l | d | l | l | d | l | l |
| **C** | u | u | u | d | l | d | l | l | l | l |
| **C** | u | u | u | d | d | d | d | d | l | d |
| **T** | u | u | d | u | d | l | d | d | l | l |
| **G** | u | u | u | u | u | d | d | u | d | l |
| **A** | u | d | u | u | u | d | u | d | u | d |

Il suffit maintenant de regarder le dernier élément $M_{nm} = 49$ pour avoir le score de l'alignement. Pour avoir l'alignement lui-même, il faut partir de $T_{nm}$ et remonter la "trace" jusqu'à arriver au 'o'. Un 'd' correspond à un *match* entre les deux séquences, 'l' à un *gap* dans la séquence $A$ et 'u' à un *gap* dans la séquence $B$. En revenant à l'exemple précédent on obtient la trace suivante :

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** | o | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **A** | &nbsp; | d | l | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; | &nbsp; |
| **T** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; |
| **G** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; |
| **A** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d |

Elle correspond à l'alignement :
```
A-CTCCTGA
ATCTCGTGA
```

**Exercice 3 :** Implémenter l'algorithme de Needlman et Wunsch. Il prendra en paramètre deux séquences et une matrice de similarité et retournera leur alignement. Tester le avec les séquences "ACTCCTGA" et "ATCTCGTGA".

In [4]:
def needlman_wunsch(similarity_matrix,a1,a2):
    matrice_score = [ [ 0 for i in range(len(a1)+2) ] for j in range(len(a2)+2) ] # Colonnes puis lignes
    matrice_traceback = [ [ 0 for i in range(len(a1)+2) ] for j in range(len(a2)+2) ] # Colonnes puis lignes


    matrice_score[0][1] = '-'
    matrice_score[1][0] = '-'
    matrice_traceback[0][1] = '-'
    matrice_traceback[1][0] = '-'

    for i in range (2,len(a1)+2):
        matrice_score[0][i] = list(a1)[i-2]
        matrice_traceback[0][i] = list(a1)[i-2]
    for i2 in range (2,len(a2)+2):
        matrice_score[i2][0] = list(a2)[i2-2]
        matrice_traceback[i2][0] = list(a2)[i2-2]



    # Initialisation valeurs gap


    for col in range(1,len(a1)+2):
        matrice_score[1][col] = -5*(col-1)
        matrice_traceback[1][col]='l'
    for lig in range(1,len(a2)+2):
        matrice_score[lig][1] = -5*(lig-1)
        matrice_traceback[lig][1] = 'u'

    for elemc in range(2,len(a1)+2): 
        for eleml in range(2,len(a2)+2):
            letter1 = matrice_score[0][elemc]
            letter2 = matrice_score[eleml][0]

            m1 = matrice_score[eleml-1][elemc-1]+similarity_matrix.values[similarity_matrix.letters.index(letter1)][similarity_matrix.letters.index(letter2)]
            m2 = matrice_score[eleml-1][elemc] + similarity_matrix.values[similarity_matrix.letters.index(letter1)][similarity_matrix.letters.index('-')]
            m3 = matrice_score[eleml][elemc-1] + similarity_matrix.values[similarity_matrix.letters.index(letter2)][similarity_matrix.letters.index('-')]


            matrice_score[eleml][elemc] = max(matrice_score[eleml-1][elemc-1]+
                            similarity_matrix.values[similarity_matrix.letters.index(letter1)][similarity_matrix.letters.index(letter2)] ,
                            matrice_score[eleml-1][elemc] + 
                            similarity_matrix.values[similarity_matrix.letters.index(letter1)][similarity_matrix.letters.index('-')] ,
                
            matrice_score[eleml][elemc-1] + 
                            similarity_matrix.values[similarity_matrix.letters.index(letter2)][similarity_matrix.letters.index('-')])

            ma = matrice_score[eleml][elemc]

            if(ma==m1):
                matrice_traceback[eleml][elemc] = 'd'
            elif(ma==m2):
                matrice_traceback[eleml][elemc] = 'u'
            else:
                matrice_traceback[eleml][elemc] = 'l'

    print('Matrice de score : \n')
    print(np.array(matrice_score))  
    print('\nMatrice de traceback : \n')
    print(np.array(matrice_traceback))  


needlman_wunsch(similarity_matrix, 'ATCTCGTGA', 'ACTCCTGA')

Matrice de score : 

[['0' '-' 'A' 'T' 'C' 'T' 'C' 'G' 'T' 'G' 'A']
 ['-' '0' '-5' '-10' '-15' '-20' '-25' '-30' '-35' '-40' '-45']
 ['A' '-5' '10.0' '5.0' '0.0' '-5.0' '-10.0' '-15.0' '-20.0' '-25.0'
  '-30.0']
 ['C' '-10' '5.0' '7.0' '12.0' '7.0' '2.0' '-3.0' '-8.0' '-13.0' '-18.0']
 ['T' '-15' '0.0' '13.0' '8.0' '20.0' '15.0' '10.0' '5.0' '0.0' '-5.0']
 ['C' '-20' '-5.0' '8.0' '20.0' '15.0' '27.0' '22.0' '17.0' '12.0' '7.0']
 ['C' '-25' '-10.0' '3.0' '15.0' '17.0' '22.0' '22.0' '19.0' '14.0'
  '11.0']
 ['T' '-30' '-15.0' '-2.0' '10.0' '23.0' '18.0' '22.0' '30.0' '25.0'
  '20.0']
 ['G' '-35' '-20.0' '-7.0' '5.0' '18.0' '18.0' '27.0' '25.0' '39.0'
  '34.0']
 ['A' '-40' '-25.0' '-12.0' '0.0' '13.0' '17.0' '22.0' '23.0' '34.0'
  '49.0']]

Matrice de traceback : 

[['0' '-' 'A' 'T' 'C' 'T' 'C' 'G' 'T' 'G' 'A']
 ['-' 'u' 'l' 'l' 'l' 'l' 'l' 'l' 'l' 'l' 'l']
 ['A' 'u' 'd' 'l' 'l' 'l' 'l' 'l' 'l' 'l' 'd']
 ['C' 'u' 'u' 'd' 'd' 'l' 'd' 'l' 'l' 'l' 'l']
 ['T' 'u' 'u' 'd' 'l' 'd' 'l' 'l' 'd' '

##### ----
## Matrice de distance

Dans le cas de séquences très proches, on estime que la distance évolutive réelle entre les séquences est proche de la p-distance qui est simplement le nombre de substitution dans l'alignement sur le nombre total de nucléotide. Pour simplifier, on ignore les positions alignées à des gaps. On applique ensuite la correction de Jukes-Cantor afin de prendre en compte le phénomène de saturation (un même site peut muter plusieurs fois au cours du temps). Sa formule est $-(\frac{3}{4})\ln(1-(\frac{4}{3})\times \textit{p-distance})$.

**Exercice 4 :** Implémenter la fonction retournant la matrice de distance à partir d'un dictionnaire de séquences. 

In [5]:
# À remplir

def distance(s1, s2):
    if len(s1) != len(s2):
        raise ValueError("Strand lengths are not equal!")
    return sum(ch1 != ch2 and ch1!='-' and ch2!='-' for ch1,ch2 in zip(s1,s2))
    
def matrice_distance(dicti):
    taille = len(dicti)
    matrice_distance = [ [ 0 for i in range(taille) ] for j in range(taille) ] # col puis lig
    
    liste_sequences = list(dicti.values())
    
    #for valeur in dicti.values():
    for i in range(0,taille): # ligne
        for j in range(0,taille): # colonne
            matrice_distance[i][j] = round((-3/4) * np.log(1-(4/3)*(distance(liste_sequences[i-1], liste_sequences[j-1])/len(liste_sequences[i-1]))), 2)            
    return np.array(matrice_distance)

print(matrice_distance(monDi))

[[-0.    0.26  0.23  0.24  0.24  0.24  0.26  0.27  0.26  0.26  0.26  0.23
   0.23  0.25  0.27  0.22  0.22  0.12  0.07]
 [ 0.26 -0.    0.16  0.16  0.15  0.19  0.13  0.16  0.16  0.16  0.17  0.18
   0.14  0.15  0.14  0.24  0.24  0.24  0.26]
 [ 0.23  0.16 -0.    0.18  0.18  0.21  0.18  0.22  0.2   0.21  0.21  0.21
   0.2   0.23  0.21  0.25  0.25  0.27  0.27]
 [ 0.24  0.16  0.18 -0.    0.18  0.18  0.1   0.11  0.13  0.12  0.08  0.06
   0.12  0.16  0.16  0.25  0.25  0.24  0.28]
 [ 0.24  0.15  0.18  0.18 -0.    0.13  0.15  0.18  0.18  0.18  0.19  0.21
   0.18  0.2   0.2   0.25  0.25  0.25  0.27]
 [ 0.24  0.19  0.21  0.18  0.13 -0.    0.17  0.18  0.19  0.18  0.16  0.22
   0.21  0.21  0.21  0.26  0.26  0.25  0.26]
 [ 0.26  0.13  0.18  0.1   0.15  0.17 -0.    0.1   0.12  0.11  0.13  0.13
   0.14  0.14  0.15  0.25  0.25  0.23  0.27]
 [ 0.27  0.16  0.22  0.11  0.18  0.18  0.1  -0.    0.02  0.01  0.11  0.14
   0.12  0.15  0.15  0.25  0.25  0.24  0.3 ]
 [ 0.26  0.16  0.2   0.13  0.18  0.19  0.12  0.0

------
## Construction d'un arbre avec UPGMA

Grâce aux mesures de distances entre les séquences, on peut maintenant de construire l'arbre phylogénétique des globines. Vous allez devoir pour cela implémenter l'algorithme UPGMA (*unweighted pair group method with arithmetic mean*) qui, malgré son nom compliqué, est l'une des méthodes les plus simples pour la construction d'arbre.

### Le format Newick

Le format Newick est l'un des formats utilisé en phylogénie pour représenter un arbre sous la forme d'une chaine de caractère. Le principe est simple, les groupes ayant la même racine sont écrit entre parenthèses et séparés par des virgules. Un groupe peut être soit une feuille de l'arbre (dans notre cas une séquence), soit un autre groupe. La longueur de la branche de chaque groupe est écrite après un double point et l'arbre est terminé par un point virgule. Pour afficher l'arbre on peut utiliser les fonction du package ETE : 

In [6]:
from ete3 import Tree, TreeStyle # ligne

newick_tree = '((A:2,(B:2,C:3):5):5,D:4);' 
t = Tree(newick_tree)
ts = TreeStyle()
ts.show_branch_length = True
t.render('%%inline', w=183, units='mm', tree_style=ts)

ModuleNotFoundError: No module named 'ete3'

**Exercice 5 :** Reécriver l'arbre suivant au format Newick puis afficher-le. Les nombres correspondent aux longueurs des branches :
![](tree.png)

In [ ]:
newick_tree = '((A:10,B:8,C:7):5,(D:9,(E:5,F:5):2):2);' 
t = Tree(newick_tree)
ts = TreeStyle()
ts.show_branch_length = True
t.render('%%inline', w=183, units='mm', tree_style=ts)

**Exercice 6 :** Expliquer la relation de parenté entre $A$, $B$ et $C$ ? Qu'elles sont les hypothèses qui pourraient expliquer ce type d'embranchement dans un arbre ? Donner une réponse détaillée.

Réponse : C'est un groupe polyphylétique puisque l'on considère A,B et C mais pas leur ancêtre commun. On pourrait expliquer ce type d'embranchement dans un arbre par une espèce qui a donné des descendants A, B et C dans plusieurs environnements, ceux-ci se sont par la suite adaptés à chacun de leur environnement, formant ainsi une spécialisation dans l'arbre.

### UPGMA

L'algorithme UPGMA se base sur la matrice de distance entre les séquences. À chaque itération, les séquences avec la distance la plus faible sont regroupées puis une nouvelle matrice de distance est calculée avec le nouveau groupe. Cette étape est répétée jusqu'à n'avoir plus qu'un seul groupe. Par exemple, avec la matrice de distance entre les séquences $A$, $B$, $C$ et $D$ suivante :

|   &nbsp;   | A | B | C | D |
|   -   | - | - | - | - |
| **A** | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **B** | 4 | &nbsp; | &nbsp; | &nbsp; |
| **C** | 8 | 8 | &nbsp; | &nbsp; |
| **D** | 2 | 4 | 8 | &nbsp; |

Les séquences $A$ et $D$ sont les plus proches ($distance(A,D)=2$). On les regroupe et on met à jour la matrice :

|   &nbsp;   | (A, D) | B | C |
|   -   | - | - | - |
| **(A, D)** | &nbsp; | &nbsp; | &nbsp; |
| **B** | 4 | &nbsp; | &nbsp; |
| **C** | 8 | 8 | &nbsp; | &nbsp; |

On regroupe maintenant $(A,D)$ et $B$ ($distance((A,D),B) = 4$) :

|   &nbsp;   | ((A, D), B) | C |
|   -   | - | - |
| **((A, D), B)** | &nbsp; | &nbsp; |
| **C** | 8 | &nbsp; |

Important : les nouvelles distances sont calculées en moyennant les distances entre les membres du nouveau groupe et des groupes non modifiés pondéré par le nombre d'UTOs dans chaque groupe. Avec $i$ et $j$ les deux membres du groupe nouvellement formé et k les groupes restant : $d_{ij,k} = \frac{n_id_{ik}}{n_i + n_j}+ \frac{n_jd_{jk}}{n_i + n_j}$. Par exemple avec la distance entre $((A, D), B)$ et $C$:

$distance(((A, D), B), C) = (distance((A, D), C)*2 + distance(B, C)) \mathbin{/} 3 = (8*2 + 8) \mathbin{/} 3 = 8 $.

L'arbre final écrit dans le format Newick est : $((A, D), B), C);$ 

Et avec les distances : $((A:1, D:1):1, B:2):2, C:4);$ 

**Exercice 7 :** Implémenter une version d'UPGMA qui calcule l'arbre au format Newick **avec les distances** puis appliquer votre algorithme aux données. 

In [88]:
import numpy as np

# on rencontre une parenthèse ( on ajoute +1 et -1 quand )
# si on arrive à 0 ça veut dire qu'on a fini le premier bloc


def lire_separateur(seq): # fonction nickel
    # on construit en fonction de si c'est une feuille ou composée déjà existante
    # si on veut le faire avec les noms de séquences les parenthèses seront trompeuses donc à voir
    #print(seq)
    
    partie_seq1 = seq
    partie_seq2 = seq
    
    '''
    if(len(seq)==3): # ex : A,B
        # cas feuille simple
        partie_seq1 = seq1
        partie_seq2 = seq2
     
    else: # ex : (H,J),I
        # cas composé, let's gooo 
        # on compte le nbr n de parenthèses ouvrantes d'affilée, on s'arrête juste après la n-1 parenthèse fermante pour séparer en deux et lire 
        # p-ê changer de stratégie en ne comptant pas les parenthèses du début
    '''
    if(len(seq)>3):
        compteur_parentheses = 0
        for e in range(len(seq)) :
            if(e==0): # premier tour le compteur sera forcément à 0
                if(seq[e]=='('):
                    compteur_parentheses+=1
            elif(compteur_parentheses!=0):
                if(seq[e]=='('):
                    compteur_parentheses+=1
                elif(seq[e]==')'):
                    compteur_parentheses-=1
            elif(compteur_parentheses==0):
                partie_seq1 = "".join(seq[:e])   
                partie_seq2 = "".join(seq[e+1:])
                break
    return partie_seq1,partie_seq2 

def calcul_distance(md,s1,s2):
    pass

def min_matrice(md):
    dmin = np.nanmin(md)
    i,j = np.where(md == dmin)
    return dmin,i[0],j[0]   

def upgma(md): #plutôt avoir des anciens rmin et cmin plutôt que découper
    #noms_sequences = tuple(list(monDi.keys())) # tuple pour modifier plus facilement les données
    lettres = tuple(list('ABCDEFGHIJKLMNOPQRS'))
    md[md == 0] = np.nan
    
    # il faut garder une trace de l'ancienne matrice pour ensuite calculer la distance avec les données lues sur celle-ci, après lecture
    mdv = md
    mda = md
        
    lettresa = lettres
    lettresv = lettres
    
    print(lettres)
    
    while not np.isnan(mdv).all():
                            
        dmin_a,rmin_a,cmin_a = min_matrice(mda)  
        #dmin,rmin,cmin = min_matrice(mdv)
        
        mdv = np.delete(mda, cmin_a, axis = 0) # row
        mdv = np.delete(mdv, cmin_a, axis = 1) # col
                
        dmin_v,rmin_v,cmin_v = min_matrice(mdv)   
        
        
        if(cmin_a < rmin_a):
            rmin_a,cmin_a = cmin_a,rmin_a
        
        for col in range(0, mdv.shape[1]):
            if(rmin_a != cmin_a):
                mdv[rmin_v, col] = (mda[rmin_a][col] + mda[cmin_a, col])/2
            
        for row in range(0, mdv.shape[0]):
            if(rmin_a != cmin_a):
                mdv[row, cmin_v] = (mda[row, rmin_a] + mda[row, cmin_a])/2
        
        #print(mdv)
        
        # lettres sert juste à afficher
        lettresv = ['('+lettresa[rmin_a]+','+lettresa[cmin_a]+')' if lettresa[e] == lettresa[rmin_a] else lettresa[e] for e in range(0,len(lettresa))]
        lettresv.remove(lettresv[cmin_a])
        
        #print(lettresv)
        #noms_sequences = ['('+noms_sequences[rmin]+','+noms_sequences[cmin]+')' if noms_sequences[e] == noms_sequences[rmin] else noms_sequences[e] for e in range(0,len(noms_sequences))]
        #noms_sequences.remove(noms_sequences[cmin])
        mda = mdv

        lettresa = lettresv
        
        #print(rmin_a,cmin_a)
        #print(lettresv)
        #print(mdv)
        
    return 0

upgma(matrice_distance(monDi))

('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S')
7 9
['A', 'B', 'C', 'D', 'E', 'F', 'G', '(H,J)', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S']
[[  nan 0.26  0.23  0.24  0.24  0.24  0.26  0.27  0.265 0.26  0.23  0.23
  0.25  0.27  0.22  0.22  0.12  0.07 ]
 [0.26    nan 0.16  0.16  0.15  0.19  0.13  0.16  0.16  0.17  0.18  0.14
  0.15  0.14  0.24  0.24  0.24  0.26 ]
 [0.23  0.16    nan 0.18  0.18  0.21  0.18  0.22  0.215 0.21  0.21  0.2
  0.23  0.21  0.25  0.25  0.27  0.27 ]
 [0.24  0.16  0.18    nan 0.18  0.18  0.1   0.11  0.115 0.08  0.06  0.12
  0.16  0.16  0.25  0.25  0.24  0.28 ]
 [0.24  0.15  0.18  0.18    nan 0.13  0.15  0.18  0.18  0.19  0.21  0.18
  0.2   0.2   0.25  0.25  0.25  0.27 ]
 [0.24  0.19  0.21  0.18  0.13    nan 0.17  0.18  0.18  0.16  0.22  0.21
  0.21  0.21  0.26  0.26  0.25  0.26 ]
 [0.26  0.13  0.18  0.1   0.15  0.17    nan 0.1   0.105 0.13  0.13  0.14
  0.14  0.15  0.25  0.25  0.23  0.27 ]
 [0.265 0.16  0.215 0.

0

In [ ]:
import numpy as np

# on rencontre une parenthèse ( on ajoute +1 et -1 quand )
# si on arrive à 0 ça veut dire qu'on a fini le premier bloc


def lire_separateur(seq): # fonction nickel
    # on construit en fonction de si c'est une feuille ou composée déjà existante
    # si on veut le faire avec les noms de séquences les parenthèses seront trompeuses donc à voir
    #print(seq)
    
    partie_seq1 = seq
    partie_seq2 = seq
    
    '''
    if(len(seq)==2): # ex : A,B
        # cas feuille simple
        partie_seq1 = seq1
        partie_seq2 = seq2
     
    else: # ex : (H,J),I
        # cas composé, let's gooo 
        # on compte le nbr n de parenthèses ouvrantes d'affilée, on s'arrête juste après la n-1 parenthèse fermante pour séparer en deux et lire 
        # p-ê changer de stratégie en ne comptant pas les parenthèses du début
    '''
    if(len(seq)>=3):
        compteur_parentheses = 0
        for e in range(len(seq)) :
            if(e==0): # premier tour le compteur sera forcément à 0
                if(seq[e]=='('):
                    compteur_parentheses+=1
            elif(compteur_parentheses!=0):
                if(seq[e]=='('):
                    compteur_parentheses+=1
                elif(seq[e]==')'):
                    compteur_parentheses-=1
            elif(compteur_parentheses==0):
                partie_seq1 = "".join(seq[:e])   
                partie_seq2 = "".join(seq[e+1:])
                break
    return partie_seq1,partie_seq2 

def calcul_distance(md,s1,s2):
    pass

def upgma(md):
    arbre = ''
    #noms_sequences = tuple(list(monDi.keys())) # tuple pour modifier plus facilement les données
    lettres = list('ABCDEFGHIJKLMNOPQRS')
    md[md == 0] = np.nan
    
    # il faut garder une trace de l'ancienne matrice pour ensuite calculer la distance avec les données lues sur celle-ci, après lecture
    mdv = md
    mda = md
    
    lettresa = tuple(lettres.copy())
    lettresv = tuple(lettres.copy())
    
    print('avant modification :',lettres)
    while not np.isnan(mdv).all():
                            
        dmin = np.nanmin(mda)
        i,j = np.where(mda == dmin)
        rmin,cmin = i[0],j[0]   

        
        mdv = np.delete(mda, cmin, axis = 0) # row
        mdv = np.delete(mdv, cmin, axis = 1) # col
        
        #on va passer à calcul_distance s1 et s2
        
        seq = list(lettresa[rmin][1:])[:-1]
        #print(seq)
        s1,s2 = lire_separateur(seq)
        print(s1)
        #print(lettresa.index(s1))
        
        for col in mdv:
            for row in mdv: #((A,B),C) = ((A,C)+(B,C))/2
                pass
        #       mdv[lettres.index(rmin), col] = (mda[lettres.index(s1)][col]+mda[lettres.index(s2)][col])/2 # peu importe si c'est distance de A à C ou C à A
        #       mdv[row,lettres.index(rmin)] = (mda[row][lettres.index(s1)]+mda[row][lettres.index(s2)])/2
        
        
        lettresv = ['('+lettresa[rmin]+','+lettresa[cmin]+')' if lettresa[e] == lettresa[rmin] else lettresa[e] for e in range(0,len(lettresa))]
        lettresv.remove(lettresv[cmin])
        
        #noms_sequences = ['('+noms_sequences[rmin]+','+noms_sequences[cmin]+')' if noms_sequences[e] == noms_sequences[rmin] else noms_sequences[e] for e in range(0,len(noms_sequences))]
        #noms_sequences.remove(noms_sequences[cmin])
        mda = mdv

        lettresa = lettresv
        print('lettres : ',lettresv)
    return 0 #arbre[:-1]+');'

upgma(matrice_distance(monDi))

In [47]:
# A Quick Implementation of UPGMA (Unweighted Pair Group Method with Arithmetic Mean)

# lowest_cell:
#   Locates the smallest cell in the table
def lowest_cell(table):
    # Set default to infinity
    min_cell = float("inf")
    x, y = -1, -1

    # Go through every cell, looking for the lowest
    for i in range(len(table)):
        for j in range(len(table[i])):
            if table[i][j] < min_cell:
                min_cell = table[i][j]
                x, y = i, j

    # Return the x, y co-ordinate of cell
    return x, y


# join_labels:
#   Combines two labels in a list of labels
def join_labels(labels, a, b):
    # Swap if the indices are not ordered
    if b < a:
        a, b = b, a

    # Join the labels in the first index
    labels[a] = "(" + labels[a] + "," + labels[b] + ")"

    # Remove the (now redundant) label in the second index
    del labels[b]


# join_table:
#   Joins the entries of a table on the cell (a, b) by averaging their data entries
def join_table(table, a, b):
    # Swap if the indices are not ordered
    if b < a:
        a, b = b, a

    # For the lower index, reconstruct the entire row (A, i), where i < A
    row = []
    for i in range(0, a):
        row.append((table[a][i] + table[b][i])/2)
    table[a] = row
    
    # Then, reconstruct the entire column (i, A), where i > A
    #   Note: Since the matrix is lower triangular, row b only contains values for indices < b
    for i in range(a+1, b):
        table[i][a] = (table[i][a]+table[b][i])/2
        
    #   We get the rest of the values from row i
    for i in range(b+1, len(table)):
        table[i][a] = (table[i][a]+table[i][b])/2
        # Remove the (now redundant) second index column entry
        del table[i][b]

    # Remove the (now redundant) second index row
    del table[b]


# UPGMA:
#   Runs the UPGMA algorithm on a labelled table
def UPGMA(table, labels):
    # Until all labels have been joined...
    while len(labels) > 1:
        # Locate lowest cell in the table
        x, y = lowest_cell(table)

        # Join the table on the cell co-ordinates
        join_table(table, x, y)

        # Update the labels accordingly
        join_labels(labels, x, y)

    # Return the final label
    return labels[0]



## A test using an example calculation from http://www.nmsr.org/upgma.htm

# alpha_labels:
#   Makes labels from a starting letter to an ending letter
def alpha_labels(start, end):
    labels = []
    for i in range(ord(start), ord(end)+1):
        labels.append(chr(i))
    return labels

# Test table data and corresponding labels
M_labels = alpha_labels("A", "G")   #A through G
M = [
    [],                         #A
    [19],                       #B
    [27, 31],                   #C
    [8, 18, 26],                #D
    [33, 36, 41, 31],           #E
    [18, 1, 32, 17, 35],        #F
    [13, 13, 29, 14, 28, 12]    #G
    ]

UPGMA(M, M_labels)

'((((A,D),((B,F),G)),C),E)'

**Exercice 8 :** Quelles sont les hypothèses faites par UPGMA ? Semblent-elles respectées dans le cas présent ?

Réponse :  
- La même quantité d’évolution s’est produite dans chaque lignée évolutive depuis leur ancêtre commun à toutes.
- Les distances évolutives entre chaque feuille et la racine sont égales.
- La racine est placée au point de l’arbre équidistant de toutes les feuilles.

----
## Enracinement de l'arbre

Après avoir utilisé UPGMA pour réaliser votre arbre, l'enracinement s'est normalement fait au poids moyen. 

**Exercice 9 :** Quelle autre méthode est-il possible d'utiliser pour enraciner un arbre ? Pouvez-vous l'utilisez ici ? Si oui, afficher le nouvel arbre.

Réponse : On peut utiliser l'enracinement avec un groupe extérieur.

----
## Neighbor-joining

Le neighbor-joining est un autre algorithme permettant de calculer un arbre phylogénique à partir d'une matrice de distance. Il a l'avantage de faire moins d'hypothèse qu'UPGMA sur les données (elles ne sont plus forcément ultramétrique) et il donne donc de meilleurs arbres dans presque tout les cas. Vous trouverez un example d'application de cet algorithme [ici](http://www.evolution-textbook.org/content/free/tables/Ch_27/T11_EVOW_Ch27.pdf).

**Exercice 10 :** Implémenter l'algorithme du neighbor-joining, appliquer-le aux données puis enraciner l'arbre.

In [ ]:
# À remplir

----
## Bootstrap

Le bootstrap est une méthode permettant de calculer la robustesse des branches d'un arbre. Il sagit de recréer un jeu de données artificiel en tirant n positions dans l'alignement des séquences avec remise, n étant la longueur de l'alignement. On recréer un arbre à partir de ces nouvelles données puis on le compare avec l'arbre obtenu avec les données réelles. Si une branche de l'arbre réelle est présente dans l'arbre artificiel, son bootstrap augmente de 1. On répètera la procédure 100 fois afin d'avoir un score sur 100 pour chaque branche.

**Exercice 11 :** Calculer le bootstrap des branches des arbres obtenus avec UPGMA et neighbor-joining.

In [ ]:
# À remplir

----
## Conclusion

**Exercice 12 :** Quelles sont vos conclusions par rapport à l'arbre phylogénique de _smilodon_, _homotherium_ et _M. trumani_ ? Comparer les deux méthodes. Comment expliquer les caractéristiques morphologiques similaires entre ces trois espèces ? Une réponse détaillée est attendue.

Réponse :